In [ ]:
# 1-) Yeni Urunlerin Sektorleri.xlsx

import pandas as pd

def load_and_process_new_products():
    # Kullanacağımız yılları tanımlıyoruz
    years = [2002, 2007, 2012, 2017, 2022]
    file_paths = {
        2002: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS02_V202401\product_codes_HS02_V202401.csv",
        2007: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS07_V202401\product_codes_HS07_V202401.csv",
        2012: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\product_codes_HS12_V202401.csv",
        2017: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\product_codes_HS17_V202401.csv",
        2022: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS22_V202401\product_codes_HS22_V202401.csv"
    }
    
    # Her yıl için ürün kodlarını ekliyoruz
    dfs = {year: pd.read_csv(file_paths[year]) for year in years}
    product_sets = {year: set(dfs[year]['code']) for year in years}

    # Yeni ürünleri buluyoruz
    new_products_dfs = []
    for i in range(1, len(years)):
        year = years[i]
        prev_year = years[i - 1]
        new_products = product_sets[year] - product_sets[prev_year]
        
        # Yıl kolonunu ekliyoruz
        new_df = dfs[year][dfs[year]['code'].isin(new_products)].copy()
        new_df['Yıl'] = year
        new_products_dfs.append(new_df)
    
    # Bütün yıllardaki yeni ürünleri tek dataframede topluyoruz
    combined_new_products = pd.concat(new_products_dfs, ignore_index=True)
    
    # 2 basamaklı HS kodunu hesaplıyoruz
    combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['code'] // 10000
    
    # Sektör kodlarını ve isimilerini kullanarak birleştiriyoruz
    atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')
    atlas_df['hs2'] = atlas_df['hs2'].astype(str).str.zfill(2)
    combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2)
    combined_new_products = combined_new_products.merge(
        atlas_df[['hs2', 'hs2_name', 'sector']],
        left_on='2 Basamaklı HS Kodu',
        right_on='hs2',
        how='left'
    ).rename(columns={
        'code': '6 Basamaklı HS Kodu',
        'hs2_name': '2 Basamaklı HS İsmi',
        'sector': 'Sektör İsmi'
    }).drop(columns=['hs2'])

    # Yıl sütununu en başa alıyoruz
    cols = ['Yıl'] + [col for col in combined_new_products.columns if col != 'Yıl']
    combined_new_products = combined_new_products[cols]
    return combined_new_products

# Fonksiyonu uyguluyoruz
processed_new_products = load_and_process_new_products()

# Sadece Sektör İsmi 'Textiles' olanları filtreliyoruz
textiles_only = processed_new_products[processed_new_products['Sektör İsmi'] == 'Textiles ']

# "4 Basamaklı HS Kodu" sütunu ekliyoruz
textiles_only['4 Basamaklı HS Kodu'] = textiles_only['6 Basamaklı HS Kodu'].astype(str).str[:4]

# Kaydediyoruz
textiles_only.to_excel("1-) Yeni Urunlerin Sektorleri.xlsx", index=False)

C:\Users\oguzo\AppData\Local\Temp\ipykernel_2852\492634467.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textiles_only['4 Basamaklı HS Kodu'] = textiles_only['6 Basamaklı HS Kodu'].astype(str).str[:4]


In [ ]:
# 2-) Türkiye'nin toplam ticaret hacmi 




In [ ]:
# Her seferinde vakit kaybetmeyeyim diye dış ticaret verilerini bir data frame'e kaydediyorum

import pandas as pd

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı
dosya_yolu_format = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"
# Tüm yılları içeren DataFrame'i birleştir
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# Ülkeleri yükle
ulkeler_df = pd.read_excel('0-) Ulkeler.xlsx')  
combined_atlas_df['sector'] = [zz.strip() for zz in combined_atlas_df['sector']] 
combined_atlas_df['hs2_name'] = [zz.strip() for zz in combined_atlas_df['hs2_name']] 

# Her yılın verisini oku ve birleştir
for year in [yillar[0]]:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    df = pd.read_csv(dosya_yolu)
    df['hs2'] = df['k']//10000
    df = df.merge(combined_atlas_df)

    ### RCA HS6
    x_p = df[['t','i', 'k','v']].groupby(['t','i','k']).sum().reset_index()
    x_p.columns = ['t','i','k','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'k','v']].groupby(['t','k']).sum().reset_index()
    w_p.columns = ['t','k','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_HS6'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','k','RCA_HS6']])

    ### RCA HS2

    x_p = df[['t','i', 'hs2','v']].groupby(['t','i','hs2']).sum().reset_index()
    x_p.columns = ['t','i','hs2','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'hs2','v']].groupby(['t','hs2']).sum().reset_index()
    w_p.columns = ['t','hs2','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_HS2'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','hs2','RCA_HS2']])


    ### RCA HS4

    x_p = df[['t','i', 'sector','v']].groupby(['t','i','sector']).sum().reset_index()
    x_p.columns = ['t','i','sector','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'sector','v']].groupby(['t','sector']).sum().reset_index()
    w_p.columns = ['t','sector','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_sector'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','sector','RCA_sector']])

    df = df[df['i'].isin(ulkeler_df['country_code'])]